In [1]:
import numpy  as np
import sympy  as sym

import matplotlib.pyplot as plt
import scipy.sparse      as sparse

import utilitis_FEEC.evaluation as eva
import utilitis_FEEC.bsplines   as bsp

import utilitis_FEEC.projectors_opt as proj

In [2]:
# ... define boundary conditions in each direction (True: periodic, False: else)
bc = [True, True, True]
bc_1, bc_2, bc_3 = bc


# ... number of elements and element boundaries in each direction
Nel_1 = 8
Nel_2 = 8
Nel_3 = 8

el_b_1 = np.linspace(0., 1., Nel_1 + 1)
el_b_2 = np.linspace(0., 1., Nel_2 + 1)
el_b_3 = np.linspace(0., 1., Nel_3 + 1)

# ... degree of spline basis (of the 0-forms) in each direction
p = [3, 3, 3]
p1, p2, p3 = p


# ... geometry
mapping = 'cube'


# ... number of basis functions
Nbase = [Nel_1 + p1, Nel_2 + p2, Nel_3 + p3] 
Nbase_1, Nbase_2, Nbase_3 = Nbase


# ... knot vectors
T1 = bsp.make_knots(el_b_1, p1, bc_1)
T2 = bsp.make_knots(el_b_2, p2, bc_2)
T3 = bsp.make_knots(el_b_3, p3, bc_3)    
T = [T1, T2, T3]



# ... define physical domain and mapping from logical domain
if mapping == 'annulus':
    
    # ... coordinates
    r, phi, z = sym.symbols('r, phi, z')
    q = sym.Matrix([r, phi, z])
    
    # ... mapping
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    
    F = sym.Matrix([(r*dR + R1)*sym.cos(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*phi), z])
    
elif mapping == 'torus':
    
    # ... coordinates
    r, theta, phi = sym.symbols('r, theta, phi')
    q = sym.Matrix([r, theta, phi])

    # ... mapping
    R0 = 1.5         # major radius
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    
    F = sym.Matrix([(R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.cos(2*sym.pi*phi), (R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.sin(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*theta)])
    
elif mapping == 'cube':
    
    # ... coordinates
    x, y, z = sym.symbols('x, y, z')
    q = sym.Matrix([x, y, z])
    
    # ... mapping
    Lx = 1.           # length in x
    Ly = 1.           # length in y
    Lz = 1.           # length in z
    
    F = sym.Matrix([Lx*x, Ly*y, Lz*z])
    

# ... jacobian matrix
DF = F.jacobian(q)

# ... metric tensor
G = sym.simplify(DF.transpose()*DF)

# ... inverse of metric tensor
Ginv = G.inverse()

#... square root of jacobi determinant
g = sym.simplify(G.det())
g_sqrt = sym.sqrt(g)


# ... convert mapping functions to callables
xc = sym.lambdify(q, F[0])
yc = sym.lambdify(q, F[1])
zc = sym.lambdify(q, F[2])

G = [[sym.lambdify(q, G[0, 0]), sym.lambdify(q, G[0, 1]), sym.lambdify(q, G[0, 2])], [sym.lambdify(q, G[1, 0]), sym.lambdify(q, G[1, 1]), sym.lambdify(q, G[1, 2])], [sym.lambdify(q, G[2, 0]), sym.lambdify(q, G[2, 1]), sym.lambdify(q, G[2, 2])]]
Ginv = [[sym.lambdify(q, Ginv[0, 0]), sym.lambdify(q, Ginv[0, 1]), sym.lambdify(q, Ginv[0, 2])], [sym.lambdify(q, Ginv[1, 0]), sym.lambdify(q, Ginv[1, 1]), sym.lambdify(q, Ginv[1, 2])], [sym.lambdify(q, Ginv[2, 0]), sym.lambdify(q, Ginv[2, 1]), sym.lambdify(q, Ginv[2, 2])]]
g = sym.lambdify(q, g)
g_sqrt = sym.lambdify(q, g_sqrt)

In [3]:
projectors = proj.projectors_3d(p, Nbase, T, bc)

In [4]:
projectors.assemble_V0()
projectors.assemble_V1()
projectors.assemble_V2()
projectors.assemble_V3()

AttributeError: module 'scipy.sparse' has no attribute 'linalg'

In [66]:
projectors.Nbase

[11, 11, 11]

In [57]:
# 0-form to be projected

#kind = 'Dirichlet'
kind = 'Periodic'

if kind == 'Dirichlet':
    a  = lambda x, y, z : x*np.sin(2*np.pi*x)*y**2*np.sin(2*np.pi*y)*z**3*np.sin(2*np.pi*z)
    
    a1 = lambda x, y, z : x*np.sin(2*np.pi*x)*y**2*np.sin(2*np.pi*y)*z**3*np.sin(2*np.pi*z)
    a1 = lambda x, y, z : x**3*np.sin(2*np.pi*x)*y*np.sin(2*np.pi*y)*z**2*np.sin(2*np.pi*z)
    a1 = lambda x, y, z : x**2*np.sin(2*np.pi*x)*y**3*np.sin(2*np.pi*y)*z*np.sin(2*np.pi*z)
    
elif kind == 'Periodic':
    a  = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    
    a1 = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    a2 = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    a3 = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)

In [58]:
vec0 = projectors.PI_0(a)

In [59]:
q1 = np.random.rand()
q2 = np.random.rand()
q3 = np.random.rand()


q = [np.array([q1]), np.array([q2]), np.array([q3])]

print(eva.evaluate_field_V0(vec0, q, p, Nbase, T, bc))
print('exact : ', a(q1, q2, q3))

[0.06752675]
exact :  0.06772333617314752


In [68]:
vec1 = projectors.PI_1([a1, a2, a3])

In [72]:
q1 = np.random.rand()
q2 = np.random.rand()
q3 = np.random.rand()


q = [np.array([q1]), np.array([q2]), np.array([q3])]

print(eva.evaluate_field_V1(vec1, q, p, Nbase, T, bc))
print('exact : ', [a1(q1, q2, q3), a2(q1, q2, q3), a3(q1, q2, q3)])

[array([0.58406915]), array([0.32641833]), array([-0.11405689])]
exact :  [0.5856204503431142, 0.32766451556466636, -0.1143892571252964]


In [73]:
vec2 = projectors.PI_2([a1, a2, a3])

In [77]:
q1 = np.random.rand()
q2 = np.random.rand()
q3 = np.random.rand()


q = [np.array([q1]), np.array([q2]), np.array([q3])]

print(eva.evaluate_field_V2(vec2, q, p, Nbase, T, bc))
print('exact : ', [a1(q1, q2, q3), a2(q1, q2, q3), a3(q1, q2, q3)])

[array([0.26215773]), array([0.54727261]), array([-0.3163972])]
exact :  [0.2627586244162342, 0.5455813393258996, -0.3183379830818045]


In [78]:
vec3 = projectors.PI_3(a)

In [80]:
q1 = np.random.rand()
q2 = np.random.rand()
q3 = np.random.rand()


q = [np.array([q1]), np.array([q2]), np.array([q3])]

print(eva.evaluate_field_V3(vec3, q, p, Nbase, T, bc))
print('exact : ', a(q1, q2, q3))

[-0.01279598]
exact :  -0.012040671610537398


In [10]:
# ... test projectors PI_0 and PI_1
if bc_x == True:
    Psi = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    grad_Psi_x = lambda x, y, z: 2*np.pi*np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_y = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_z = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    
else:
    Psi = lambda x, y, z : x*(1 - x)*y*(1 - y)*z*(1 - z)

    grad_Psi_x = lambda x, y, z: (1 - 2*x)*y*(1 - y)*z*(1 - z)
    grad_Psi_y = lambda x, y, z: (1 - 2*y)*x*(1 - x)*z*(1 - z)
    grad_Psi_z = lambda x, y, z: (1 - 2*z)*x*(1 - x)*y*(1 - y)



x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 0')
print(Psi(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V0(proj.PI_0(Psi, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

print('----------------------------')
print('Projector 1')
print(grad_Psi_z(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_z(proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 0
0.002768354417264192
0.0027683544172641917
----------------------------
Projector 1
-0.010558440324552196
-0.010558440324552196


In [4]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = Psi(X, Y, zplot[20])
psi = eva.evaluate_field_V0(proj.PI_0(Psi, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [8]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = grad_Psi_z(X, Y, zplot[20])
psi = eva.evaluate_field_V1_z(proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [11]:
# ... test projectors PI_1 and PI_2
if bc_x == True:
    Ex = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    Ey = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Ez = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    curl_Ex = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ey = lambda x, y, z : 2*np.pi*(np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ez = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    
else:
    Ex = lambda x, y, z: x*y*(1 - y)*z*(1 - z)
    Ey = lambda x, y, z: y*x*(1 - x)*z*(1 - z)
    Ez = lambda x, y, z: z*x*(1 - x)*y*(1 - y)

    curl_Ex = lambda x, y, z: z*x*(1 - x)*(1 - 2*y) - y*x*(1 - x)*(1 - 2*z)
    curl_Ey = lambda x, y, z: x*y*(1 - y)*(1 - 2*z) - z*(1 - 2*x)*y*(1 - y)
    curl_Ez = lambda x, y, z: y*(1 - 2*x)*z*(1 - z) - x*(1 - 2*y)*z*(1 - z)

x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 1')
print(Ex(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_x(proj.PI_1_x(Ex, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])


print('----------------------------')
print('Projector 2')
print(curl_Ez(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_z(proj.PI_2_z(curl_Ez, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 1
0.01188239626119957
0.011882396261199576
----------------------------
Projector 2
-0.07472199889748922
-0.07472199889748932


In [10]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = curl_Ez(X, Y, zplot[20])
psi = eva.evaluate_field_V2_z(proj.PI_2_z(curl_Ez, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [12]:
# ... test projectors PI_2 and PI_3
if bc_x == True:
    Bx = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    By = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Bz = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    div_B = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z))

else:
    Bx = lambda x, y, z : x*(1 - x)*y*z
    By = lambda x, y, z : y*(1 - y)*x*z
    Bz = lambda x, y, z : z*(1 - z)*x*y
    
    div_B = lambda x, y, z : y*z + x*z + x*y - 6*x*y*z
    
x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 2')
print(Bx(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_x(proj.PI_2_x(Bx, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

print('----------------------------')
print('Projector 3')
print(div_B(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V3(proj.PI_3(div_B, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 2
0.03891840038961674
0.03891840038961674
----------------------------
Projector 3
-0.5905875998969077
-0.5905875998969072


In [8]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = div_B(X, Y, zplot[20])
psi = eva.evaluate_field_V3(proj.PI_3(div_B, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [13]:
# ... check commuting diagram property (V0 --> V1)
vec_0 = GRAD.dot(proj.PI_0(Psi, p, Nbase, T, bc))

vec_1_x = proj.PI_1_x(grad_Psi_x, p, Nbase, T, bc)
vec_1_y = proj.PI_1_y(grad_Psi_y, p, Nbase, T, bc)
vec_1_z = proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc)

vec_1 = np.append(vec_1_x, np.append(vec_1_y, vec_1_z))

print('max. error :', np.abs(vec_0 - vec_1).max())

max. error : 5.2909066017292616e-17


In [14]:
# check commuting diagram property (V1 --> V2)
vec_1_x = proj.PI_1_x(Ex, p, Nbase, T, bc)
vec_1_y = proj.PI_1_y(Ey, p, Nbase, T, bc)
vec_1_z = proj.PI_1_z(Ez, p, Nbase, T, bc)

vec_1 = CURL.dot(np.append(vec_1_x, np.append(vec_1_y, vec_1_z)))

vec_2_x = proj.PI_2_x(curl_Ex, p, Nbase, T, bc)
vec_2_y = proj.PI_2_y(curl_Ey, p, Nbase, T, bc)
vec_2_z = proj.PI_2_z(curl_Ez, p, Nbase, T, bc)

vec_2 = np.append(vec_2_x, np.append(vec_2_y, vec_2_z))

print('max. error :', np.abs(vec_1 - vec_2).max())

max. error : 6.201636426617085e-17


In [15]:
# check commuting diagram property (V2 --> V3)
vec_2_x = proj.PI_2_x(Bx, p, Nbase, T, bc)
vec_2_y = proj.PI_2_y(By, p, Nbase, T, bc)
vec_2_z = proj.PI_2_z(Bz, p, Nbase, T, bc)

vec_2 = DIV.dot(np.append(vec_2_x, np.append(vec_2_y, vec_2_z)))

vec_3 = proj.PI_3(div_B, p, Nbase, T, bc)

print('max. error :', np.abs(vec_2 - vec_3).max())

max. error : 1.474514954580286e-17
